# Capstone Week3 : Segmenting and Clustering Neighborhoods in Toronto

In [48]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pandas as pd
import numpy as np

## Web scrapping the wikipedia page:https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

#### Read the Wikipedia page directly wti the pandas read_html module .. and keep first table of that page as this is the one we want 

In [57]:
tables = pd.read_html(url) # Returns list of all tables on page
# First table of that page is what we need .. 
toronto_neigh = tables[0]
#print (toronto_neigh.dtypes, toronto_neigh.shape)



#### Clean the data frame by removing the Borought with 'not assigned' value then grouping based on Postal Code and Borough to aggregate the Neighbourhood in a single line

In [65]:
# clean and format data frame :
# First remove spaces in the column name, as a good pratice 
toronto_neigh.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
toronto_neigh = toronto_neigh.loc[toronto_neigh['Borough']!='Not assigned']
#More than one neighborhood can exist in one postal code area ...  These two rows will be combined into one row with the neighborhoods separated with a comma 
toronto_grouped = toronto_neigh.groupby(['PostalCode','Borough'], as_index=False).agg(lambda x: ','.join(x))
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
mask = toronto_grouped['Neighbourhood'] == "Not assigned"
toronto_grouped.loc[mask, 'Neighbourhood'] = toronto_grouped.loc[mask, 'Borough']

print ('Dataframe shape:',toronto_grouped.shape)
display (toronto_grouped.head())

Dataframe shape: (103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Work with the coordinate Latitude Longitude, based on csv file : https://cocl.us/Geospatial_data and merge dataframes

In [67]:
# read the csv file and convert to dataframe 
import io
import requests
url = "https://cocl.us/Geospatial_data"
src = requests.get(url).content  # get raw formatted content data from that url
coord = pd.read_csv(io.StringIO(src.decode('utf-8'))) # pass to pandas read_csv a 'file-like' object the raw we got from prev step

# again rename header to remove space 
coord.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace = True)

# mergde our datafile wth the coordinate dataframe
neigh = pd.merge(toronto_grouped, coord, on='PostalCode', how='inner') # Join info on PostalCode which is our unique value, common to both dataframe

display (neigh.head())

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## Explore and cluster the neighborhoods in Toronto